Finite-Volume Method

Most popular method in computational fluid dynamics (everyone loves it)

* FVM discretizations are very general and are very flexible

* FVM gives a conservative discretization automatically by directly using the conservation laws in integral form.

Conservation:
$$\frac{\partial}{\partial t} \int_{\text{cv}} e dV + \oint_{\text{cs}} \vec{F} \cdot d\vec{A} = 0$$

Why not make the control volume itself our computational cell?

$$e_i = \frac{1}{\Delta x} int_{x_i - \Delta x/2}^{x_i + \Delta x/2} e(x,t) dx$$

If we know the flux terms at the boundaries, the general conservation law gives:

$$\frac{\partial}{\partial t} e_i + \frac{1}{\Delta x} \left[ F(x_{i + 1/2},t) - F(x_{i-1/2},t)\right] = 0$$

Godunov's Method (1959)

Uses integral form of the conservation laws and a piecewise constant representation of the solution

Use Riemann solution at each cell boundary (what a smart guy)

Need to solve Riemann from $t$ to $\Delta t$ one on each cell boundary

$$F_{i+1/2} = \frac{1}{\Delta t} \int_{t^n}^{t^{n+1}} F(e(x_{i+1/2},t)) dt$$

Rusanov flux aka Lax-Friedrichs flux

$$F_{i+1/2} = \frac{1}{2} [F(e_L) + F(e_R)] - \frac{1}{2} max |F'(e)| (e_R - e_L)$$

F'(e) is the Jacobian of the flux function and max|F'(e)| is the local propagation speed of the fastest traveling wave.

The Riemann solutions at each cell boundary don't interact if $max|F'(e)| \leq \frac{\Delta x}{\Delta t}$ which leads to:
$$F_{i+1/2} = \frac{1}{2} \left( F(e_i) + F(e_{i+1}) - \frac{\Delta x}{\Delta t} (e_{i+1} - e_i) \right)$$

In [3]:
from traffic import rho_red_light, computeF

In [5]:
%matplotlib inline
import numpy
from matplotlib import pyplot
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 16
from matplotlib import animation
from JSAnimation.IPython_display import display_animation

In [6]:
#Basic initial condition parameters
#defining grid size, time steps, CFL condition, etc...
nx = 101
nt = 30
dx = 4.0/(nx-2)

rho_in = 5.
rho_max = 10.

V_max = 1.

x = numpy.linspace(0,4,nx-1)

rho = rho_red_light(nx-1, rho_max, rho_in)

In [7]:
def animate(data):
    x = numpy.linspace(0,4,nx-1)
    y = data
    line.set_data(x,y)
    return line,

In [8]:
def godunov(rho, nt, dt, dx, rho_max, V_max):
    """Computes the solution with the Godunov scheme using the Lax-Friedrichs flux.
    
    Parameters
    ----------
    rho : array of floats
        Density at current time-step
    nt : int
        Number of time steps
    dt : float
        Time-step size
    dx : float
        Mesh spacing
    rho_max : float
        Maximum allowed car density
    V_max : float
        Speed limit
        
    Returns
    -------
    rho_n : array of floats
        Density after nt time steps at every point x
    """
    
    #initialize our results array with dimensions nt by nx
    rho_n = numpy.zeros((nt,len(rho)))
    #copy the initial u array into each row of our new array
    rho_n[:,:] = rho.copy()
    
    #setup some temporary arrays
    rho_plus = numpy.zeros_like(rho)
    rho_minus = numpy.zeros_like(rho)
    flux = numpy.zeros_like(rho)
    
    for t in range(1,nt):
        
        rho_plus[:-1] = rho[1:] #can't do i+1/2 indices so cell boundary
        rho_minus = rho.copy() #arrays at index i are at location i+1/2
        flux = 0.5 * (computeF(V_max, rho_max, rho_minus) + computeF(V_max, rho_max, rho_plus) + dx/dt * (rho_minus - rho_plus))
        rho_n[t,1:-1] = rho[1:-1] + dt/dx*(flux[:-2]-flux[1:-1])
        rho_n[t,0] = rho[0]
        rho_n[t,-1] = rho[-1]
        rho = rho_n[t].copy()
        
    return rho_n

In [9]:
sigma = 1.0
dt = sigma*dx/V_max

rho = rho_red_light(nx-1, rho_max, rho_in) # make sure that u is set to our expected initial conditions
rho_n = godunov(rho, nt, dt, dx, rho_max, V_max)